In [2]:
# ignore this cell - it makes the emphasized text red and uses the full width of the screen
from IPython.core.display import HTML
HTML('<style>em { color: red; }</style> <style>.container {width:100% !important; }</style>')

In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect("worksheet.db")

In [3]:
def qry(QUERRY):
    return pd.read_sql(QUERRY, conn)

In [4]:
pd.read_sql("SELECT * from sqlite_master", conn)

,type,name,tbl_name,rootpage,sql
0,table,hydrants,hydrants,2,"CREATE TABLE ""hydrants"" (\n""year"" INTEGER,\n ..."
1,table,trees,trees,3,"CREATE TABLE ""trees"" (\n""tree"" TEXT,\n ""x"" IN..."
2,table,species,species,4,"CREATE TABLE ""species"" (\n""code"" TEXT,\n ""spe..."


In [5]:
hydrants = qry("SELECT * FROM hydrants")
trees = qry("SELECT * FROM trees")
species = qry("SELECT * FROM species")

In [6]:
trees

,tree,x,y,species,diameter,priority
0,A,10,4,m,8,71
1,B,20,4,m,10,100
2,C,30,4,p,6,30
3,D,40,4,p,8,40
4,E,50,4,m,12,99


### 1a. *Without* running this cell - *predict* the output of the following statement

In [7]:
# trees[trees["priority"] > 90][["x", "y"]] 
#   x   y
#1  20  4
#4  50  4

### 1b. *Convert* the statement to an equivalent *SQL* querry.

In [7]:
qry("""
SELECT x, y
FROM trees
WHERE priority > 90
""")

,x,y
0,20,4
1,50,4


----
### 2a. *Predict* the output of the following *SQL* querry

In [8]:
# qry("SELECT x+y FROM trees WHERE species = 'm'")

#    x+y
# 0   14
# 1   24
# 2   54

### 2b. *Convert* the querry into an equivalent *pandas* statement.

In [10]:
trees[trees["species"]=="m"]["x"] + trees[trees["species"]=="m"]["y"]

0    14
1    24
4    54
dtype: int64

----
### 3a. *Predict* the output of the following *pandas* statements

In [13]:
# cd = species["code"][species["species"]=="maple"].iloc[0]
# trees[trees["species"] == cd]["tree"]

species
# cd == "m"
#
cd = species["code"][species["species"]=="maple"].iloc[0]
#cd

#0  A
#1  B
#4  E
trees[trees["species"] == cd]["tree"]

0    A
1    B
4    E
Name: tree, dtype: object

### 3b. *Convert* the statements into an equivalent *SQL* querry.

In [16]:
cd = qry("""
SELECT code FROM species WHERE species = 'maple'
""")["code"].iloc[0]

qry("""
SELECT tree FROM trees WHERE species = '{}'
""".format(cd))

,tree
0,A
1,B
2,E


----
### 4a. *Predict* the output of the following querry

In [24]:
#qry("SELECT species FROM trees ORDER BY priority DESC")
trees
#0 m
#1 m
#2 m
#3 p
#4 p

,tree,x,y,species,diameter,priority
0,A,10,4,m,8,71
1,B,20,4,m,10,100
2,C,30,4,p,6,30
3,D,40,4,p,8,40
4,E,50,4,m,12,99


### 4.b *Convert* the querry code to *Pandas*

In [26]:
trees.sort_values("priority", ascending = False)["species"]

1    m
4    m
0    m
3    p
2    p
Name: species, dtype: object

----
### 5a. *Predict* the output of the following code

In [28]:
list(qry("SELECT tree, priority FROM trees ORDER BY priority DESC LIMIT 1").iloc[0])
#trees
#0 B 100
#["B", 100]

['B', 100]

### 5.b *Convert* the above code to *Pandas*

In [32]:
list(trees.sort_values("priority",ascending=False)[["tree","priority"]][:1].iloc[0])

['B', 100]

----
### 6a. *Predict* the output of the following code

In [34]:
qry("""SELECT COUNT(SPECIES) AS c1,
 COUNT(DISTINCT SPECIES) as c2
 FROM trees""")
#trees
#  c1  c2
#0  5   2 

,c1,c2
0,5,2


### 6b. *Convert* the above code to *Pandas*

In [40]:
trees
len(trees["species"])
trees["species"].value_counts() #Gives a series with the frequencies of each distinct element
pd.DataFrame([{"c1":len(trees["species"]),
              "c2":len(trees["species"].value_counts())}])

,c1,c2
0,5,2


----
### 7a. *Predict* the output of the following code

In [42]:
qry("""SELECT species, COUNT(SPECIES) AS count,
 AVG(diameter) AS size
 FROM trees
 GROUP BY species ORDER BY count DESC""")
#trees

#   species  count  size
#0     m        3    10     <-- Actual result had 10.0 and 7.0
#1     p        2    7

,species,count,size
0,m,3,10.0
1,p,2,7.0


### 7b. *Convert* the above code to *Pandas*

In [78]:
vc = trees["species"].value_counts()
print(vc)
#trees.groupby("species").mean().index <-- Note that species column became the index!
print(trees.groupby("species").mean().reset_index())
gb = trees.groupby("species").mean()
pd.DataFrame({
    "species":gb.index,
    "count":vc,
    "size":gb["diameter"]
}) #ALMOST THERE but out index is labeled "species" instead
pd.DataFrame({
    #"species":gb.index,
    "count":vc,
    "size":gb["diameter"]
}).reset_index()

m    3
p    2
Name: species, dtype: int64
  species          x    y  diameter  priority
0       m  26.666667  4.0      10.0      90.0
1       p  35.000000  4.0       7.0      35.0


,species,count,size
0,m,3,10.0
1,p,2,7.0


----
# Additional Exercises: 
### *Predict* the output of the following statements

In [15]:
# qry("SELECT color, year FROM hydrants WHERE color = 'blue' ")

In [16]:
# df = qry("SELECT color, year FROM hydrants")
# df[df.color == "blue"]

In [17]:
# qry("SELECT year FROM hydrants WHERE owner='private' AND active")

In [18]:
# df = qry("SELECT year, style, active FROM hydrants")
# df[df.active == 1]["style"]

In [19]:
# hydrants["color"].value_counts()

In [20]:
# qry("""SELECT color, COUNT(*) FROM hydrants
# WHERE active GROUP BY color""")

In [21]:
# qry("""SELECT color, COUNT(*) AS count FROM hydrants
# GROUP BY color HAVING count > 1""")

In [22]:
# qry("""SELECT color, COUNT(*) AS count
# FROM hydrants WHERE year >= 2000
# GROUP BY color HAVING count < 2""")